In [ ]:
import pandas as pd

data_path = "/global/scratch/users/toniooppi/nuclear_data/cmsr_dataset_material_movement.h5"
df_base = pd.read_hdf(data_path)

In [ ]:
df = df_base.copy()

In [ ]:
df

In [8]:
df.drop(df.columns[df.columns.str.contains('^(tank_|flow_)', regex=False)], axis=1, inplace=True)

In [5]:
def convert_columns_to_dict(df):
    # Define prefixes to process
    prefixes = ['fuel_']
    exclude_cols = ['fuel_TOT_DH', 'fuel_TOT_GS', 'fuel_TOT_A']
    
    for prefix in prefixes:
        # Get all columns that start with the prefix and aren't in exclude_cols
        cols = [col for col in df.columns if col.startswith(prefix) and col not in exclude_cols]
        
        if not cols:
            continue
            
        # Create the new dictionary column name
        new_col = f"{prefix}value"
        
        # Process row by row instead of using apply
        dict_series = []
        for _, row in df[cols].iterrows():
            row_dict = {
                col[len(prefix):]: val
                for col, val in row.items()
                if val != 0
            }
            dict_series.append(row_dict)
            
        # Assign the new column
        df[new_col] = dict_series
        
        # Drop the original columns
        df = df.drop(columns=cols)
        
    return df

# Usage:
df_converted = convert_columns_to_dict(df)

In [6]:
df_converted

,IE,BU,CT,FMI,fuel_TOT_GS,fuel_TOT_DH,fuel_TOT_A,AN,SF,fuel_value
0,10.0,0.0000,0.0,0,1.794450e+12,9.92946,1.285680e+13,0.8384,0.02793,"{'F19': 1.19531, 'Na23': 0.396987, 'K39': 0.26..."
1,10.0,0.0000,0.0,1,1.794450e+12,9.92946,1.285680e+13,0.8384,0.02793,"{'F19': 1.19531, 'Na23': 0.396987, 'K39': 0.26..."
2,10.0,0.0000,0.0,2,1.794450e+12,9.92946,1.285680e+13,0.8384,0.02793,"{'F19': 1.19531, 'Na23': 0.396987, 'K39': 0.26..."
3,10.0,0.0000,0.0,3,1.794450e+12,9.92946,1.285680e+13,0.8384,0.02793,"{'F19': 1.19531, 'Na23': 0.396987, 'K39': 0.26..."
4,10.0,0.0000,0.0,4,1.794450e+12,9.92946,1.285680e+13,0.8384,0.02793,"{'F19': 1.19531, 'Na23': 0.396987, 'K39': 0.26..."
...,...,...,...,...,...,...,...,...,...,...
860995,20.0,28.8799,3650.0,30,1.280680e+17,42216.70000,5.408690e+17,462.2000,2.61300,"{'H1': 3.88216e-06, 'H2': 2.0856e-08, 'H3': 2...."
860996,20.0,28.8799,3650.0,31,1.277880e+17,42177.00000,5.401640e+17,461.4000,2.59600,"{'H1': 3.8823e-06, 'H2': 2.09909e-08, 'H3': 2...."
860997,20.0,28.8799,3650.0,32,1.274950e+17,42130.30000,5.395010e+17,461.7000,2.60500,"{'H1': 3.89707e-06, 'H2': 2.10247e-08, 'H3': 2..."
860998,20.0,28.8799,3650.0,33,1.272530e+17,42095.90000,5.387970e+17,461.5000,2.60700,"{'H1': 3.88554e-06, 'H2': 2.0904e-08, 'H3': 2...."


In [7]:
df_converted.to_csv('df_converted_fuel.csv', index=False)

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

# Parse the XML file
tree = ET.parse('global/scratch/users/toniooppi/nuclear_data/chain-endf-b8.0.xml')
root = tree.getroot()

# Create a dictionary to store nuclide half-lives
half_lives = {}

# Extract half-lives from XML
for nuclide in root.findall('nuclide'):
    name = nuclide.get('name')
    half_life = nuclide.get('half_life')
    if half_life is not None:
        half_lives[name] = float(half_life)
    else:
        half_lives[name] = 1.0  # Default value for stable nuclides

# Assuming your density dictionary looks like this
densities = {
    'H1': 1.0,
    'H2': 2.0,
    'H3': 3.0,
    'H4': 4.0,
    'H5': 5.0
}

# Convert density dictionary to DataFrame
df = pd.DataFrame.from_dict(densities, orient='index', columns=['density'])

# Add half-life column and multiply
df['half_life'] = df.index.map(half_lives)
df['result'] = df['density'] * df['half_life']

FileNotFoundError: [Errno 2] No such file or directory: 'global/scratch/users/toniooppi/nuclear_data/chain-endf-b8.0.xml'